In [233]:
import pandas as pd # pour traiter les tableaux
import shutil # pour gérer les fichiers et en particulier copier le fichier modèle
import numpy  as np

In [56]:
from frozenlist import FrozenList

In [114]:
import datetime

In [127]:
days_jours = { 'Monday' : 'Lundi',
               'Tuesday' : 'Mardi',
               'Wednesday' : 'Mercredi', 
               'Thursday' : 'Jeudi', 
               'Friday' : 'Vendredi'}

In [3]:
stat_inscrits = pd.read_excel("statistiques_inscrits.xls", skiprows=3)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [136]:
stat_inscrits['nom_classe'] = stat_inscrits.Classe.str.split(expand=True).iloc[:,[0]]
stat_inscrits['Niveau'] = stat_inscrits.Classe.str.split(expand=True).iloc[:,[1]]
stat_inscrits['Clae'] = stat_inscrits.Groupe.str.split(expand=True).iloc[:,[1]]
stat_inscrits['Nom'] = stat_inscrits.Enfant.str.split(expand=True).iloc[:,[0]]
stat_inscrits['Prenom'] = stat_inscrits.Enfant.str.split(expand=True).iloc[:,[1]]
stat_inscrits['jour'] = stat_inscrits['Début inscr.'].dt.day_name().map(lambda x : days_jours[x])
stat_inscrits['date'] = stat_inscrits['jour'] + ' ' + stat_inscrits['Début inscr.'].map(lambda x: datetime.datetime.strftime(x,'%d/%m'))


In [163]:
correspondance_jour = stat_inscrits[['Début inscr.', 'date', 'jour']].drop_duplicates().sort_values('Début inscr.') 

In [164]:
correspondance_jour[correspondance_jour.jour!='Mercredi'].date.to_list()

['Mardi 12/11', 'Jeudi 14/11', 'Vendredi 15/11']

Je séléctionne les lundi, mardi, jeudi et vendredi

In [167]:
stat_inscrits_lmjv = stat_inscrits[stat_inscrits['jour'].isin(['Lundi', 'Mardi', 'Jeudi', 'Vendredi']) ] 

Je remplis les onglets pour le lundi, mardi, jeudi et vendredi

In [340]:
date_min = stat_inscrits['Début inscr.'].min().strftime("%d-%b-%Y")
name_file = 'Suivi Des Presences Hebdomadaire-{}.xlsx'.format(date_min)
shutil.copy("Suivi Des Presences Hebdomadaire-Modele.xlsx", name_file) 

'Suivi Des Presences Hebdomadaire-12-Nov-2024.xlsx'

In [341]:
first_level = np.repeat(correspondance_jour[correspondance_jour.jour!='Mercredi'].date.to_list(),3)
first_level = np.concatenate([['',''], first_level])
second_level = np.tile(['matin','midi', 'soir'], correspondance_jour[correspondance_jour.jour!='Mercredi'].shape[0])
second_level = np.concatenate([['Enfant', 'Niveau'], second_level])
array = first_level, second_level
Classe_pivot.columns= pd.MultiIndex.from_arrays(array)

In [342]:
for nom_classe in ['Aiga', "Tèrra", "Fuòc", "Amor", "Aire"]:
    Classe = stat_inscrits_lmjv[stat_inscrits_lmjv.nom_classe==nom_classe]
    Classe = Classe[Classe.Groupe.isin(['CLAE Matin', 'CLAE Midi', 'CLAE Soir'])]
    Classe = Classe[~Classe.Groupe.isna()]
    Classe = pd.concat([Classe, pd.get_dummies(Classe.Clae)[['Matin', 'Midi', 'Soir']]], axis=1)
    Classe_pivot = Classe.pivot_table(values=['Matin', 'Midi', 'Soir'], index=['Enfant', 'Niveau'], columns=['date'], aggfunc='sum')#.swaplevel(0, 1, axis=1)
    Classe_pivot = Classe_pivot.reorder_levels([1, 0], axis=1).sort_values('date', axis=1)
    Classe_pivot = Classe_pivot[correspondance_jour[correspondance_jour.jour!='Mercredi'].date.to_list()]
    Classe_pivot = Classe_pivot.reset_index()
    Classe_pivot.columns= pd.MultiIndex.from_arrays(array)
    summary = pd.DataFrame(np.array(Classe_pivot.iloc[:,2:].sum(axis=0, ).to_list() ).reshape(1,len(correspondance_jour[correspondance_jour.jour!='Mercredi'].date.to_list()) * 3))
    Classe_pivot.replace(0,'',inplace=True)
    #Classe_pivot.columns.names = FrozenList([None, None])
    with pd.ExcelWriter(name_file, mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
        Classe_pivot.to_excel(writer, sheet_name=nom_classe, startrow=3, index_label=False)  
        summary.to_excel(writer, sheet_name=nom_classe, startrow=30, startcol = 3, index_label=False, index=False)